In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
"""
Static_Analysis_RAWDATA.7z: https://drive.google.com/file/d/1XfnQMagW-yclH-wHZZRvYJHZSBExXzZu/view?usp=sharing 
Dynamic_Analysis_Data_Part1.7z: https://drive.google.com/file/d/13rmnrPsnoqjRBflDq6e59bW_YoaDeGxx/view?usp=sharing  //train
Dynamic_Analysis_Dataset_Part2.7z: https://drive.google.com/file/d/10P5R5WtK5NOV3-KF7yBGqLcidzMZJ8Uv/view?usp=sharing  //retrain

"""


!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the Shareable link
#Ex link : https://drive.google.com/file/d/1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp/view?usp=sharing
# Get the id from the link 1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp


downloaded = drive.CreateFile({'id':'1XfnQMagW-yclH-wHZZRvYJHZSBExXzZu'})   
downloaded.GetContentFile('static.7z')       
# downloaded = drive.CreateFile({'id':'13rmnrPsnoqjRBflDq6e59bW_YoaDeGxx'})   
# downloaded.GetContentFile('dynamic1.7z')       
# downloaded = drive.CreateFile({'id':'10P5R5WtK5NOV3-KF7yBGqLcidzMZJ8Uv'})   
# downloaded.GetContentFile('dynamic2.7z')       

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install p7zip-full
!p7zip -d static.7z
# !p7zip -d dynamic1.7z
# !p7zip -d dynamic2.7z


Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1321592898 bytes (1261 MiB)

Extracting archive: static.7z
--
Path = static.7z
Type = 7z
Physical Size = 1321592898
Headers Size = 719935
Method = LZMA:23
Solid = +
Blocks = 1

  0%      0% 16 - Static_Analysis_Data/Benign/0 . 28ad901aa57/Structure_Info.txt                                

In [2]:
import os
import urllib.request, json 
import pandas as pd
import ast
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
from sklearn.preprocessing import MinMaxScaler
import random
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
sc=MinMaxScaler()


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#divide hashes folder total =5000 into 3000 train and 2000 test

In [ ]:
import os
import pandas as pd
i=0

hash=[]
string_text=[]
type_of_malware=[]
for filename in os.listdir('/content/Static_Analysis_Data/Benign'):
  hash.append(filename)
  type_of_malware.append(0)
  benign=[]
  filepath = '/content/Static_Analysis_Data/Benign/'+filename+'/String.txt'
  with open(filepath) as fp:
    j=0
    for line in enumerate(fp):
      val=line[1].strip().rstrip("\n")  
      benign.append(val)
  # benign=list(dict.fromkeys(benign))
  string_text.append(benign)
  i+=1
  if(i==3000):
    break
print(i)

3000


In [ ]:
len(benign)
# string_text=[list(t) for t in set(tuple(element) for element in string_text)]
# len(string_text)

278

In [ ]:
X_Train.columns

Index(['Unnamed: 0', 'Hash', 'String', 'Type'], dtype='object')

In [ ]:
X_Train = pd.DataFrame(hash,columns=["Hash"])

X_Train['Type']=type_of_malware

In [ ]:
X_Train['String']=string_text

ValueError: ignored

In [6]:
X_Train.to_csv('BenignTrainwithCol-Hash-String-Type.csv')

NameError: ignored

In [3]:
X_Train=pd.read_csv('BenignTrainwithCol-Hash-String-Type.csv')

In [9]:
X_Train.shape

(3000, 4)

In [ ]:
###NOW TEST BENIGN

In [ ]:
import os
i=0
hash=[]
string_text=[]
for filename in os.listdir('/content/Static_Analysis_Data/Benign'):
  i+=1
  if(i<3000):
    continue
  hash.append(filename)  
  benign=[]
  filepath = '/content/Static_Analysis_Data/Benign/'+filename+'/String.txt'
  with open(filepath) as fp:
    j=0
    for line in enumerate(fp):
      j+=1
      if(j==1):
        continue
      val=line[1].strip().rstrip("\n")  
      benign.append(val)
  string_text.append(benign)
print(i)
X_Test = pd.DataFrame(hash,columns=["Hash"])

5000


In [ ]:
X_Test = pd.DataFrame(hash,columns=["Hash"])
X_Test['String']=string_text
X_Test.to_csv('BenignTestwithCol-Hash-String.csv')

In [4]:
X_Test=pd.read_csv('BenignTestwithCol-Hash-String.csv')

In [ ]:
i=0
import os
malware=[]
hash=[]
string_text=[]
type_of_malware=[]
for filename in os.listdir('/content/Static_Analysis_Data/Malware'):  
  for filename2 in os.listdir('/content/Static_Analysis_Data/Malware/'+filename):
    hash.append(filename2)
    type_of_malware.append(1)
    malware=[]
    filepath = '/content/Static_Analysis_Data/Malware/'+filename+'/'+filename2+'/String.txt'
    with open(filepath) as fp:
      j=0
      for line in enumerate(fp):
        j+=1
        if(j==1):
          continue
        val=line[1].strip().rstrip("\n")  
        malware.append(val)
    string_text.append(malware)
  i+=1

print(i)



6


In [ ]:
X2 = pd.DataFrame(hash,columns=["Hash"])
X2['String']=string_text
X2['Type']=type_of_malware

X2.to_csv('MalwareCompleteDataSet.csv')

In [ ]:
X2.shape

(5000, 3)

In [5]:
X2=pd.read_csv('MalwareCompleteDataSet.csv')

In [6]:
X2_Train=X2.iloc[:3000,:]
X2_Test=X2.iloc[3000:,:]

In [7]:
X2_Test.drop(['Type'],axis=1,inplace=True)
X2 = X2.loc[:, ~ X2.columns.str.contains('^Unnamed')]
X_Train = X_Train.loc[:, ~ X_Train.columns.str.contains('^Unnamed')]
X_Test = X_Test.loc[:, ~ X_Test.columns.str.contains('^Unnamed')]
X2_Train = X2_Train.loc[:, ~ X2_Train.columns.str.contains('^Unnamed')]
X2_Test = X2_Test.loc[:, ~ X2_Test.columns.str.contains('^Unnamed')]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
X2.shape

(5000, 3)

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#WE HAVE
"""
only strings now

X_Train Benign
X_Test

X2_Train  Malware
X2_Test

"""

In [8]:


Train_db=pd.merge(X_Train,X2_Train,on=['Hash','Type'],how='left')

Test_db=pd.merge(X_Test,X2_Test,on=['Hash'],how='left')

In [9]:
copy_test=pd.merge(X_Test,X2_Test,on=['Hash'],how='left')

In [ ]:
Train_db.head()

,Hash,String_x,Type
2627,1957,1074,0
745,814,1986,0
2815,2269,2001,0
1275,1980,578,0
2579,1013,2152,0


In [10]:
Train_db.info()
Train_db= Train_db.sample(frac=1).reset_index(drop=True)
Test_db= Test_db.sample(frac=1).reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Hash      3000 non-null   object
 1   String_x  3000 non-null   object
 2   Type      3000 non-null   int64 
 3   String_y  0 non-null      object
dtypes: int64(1), object(3)
memory usage: 117.2+ KB


In [25]:
Test_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2001 entries, 0 to 2000
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Hash      2001 non-null   object
 1   String_x  2001 non-null   object
dtypes: object(2)
memory usage: 46.9+ KB


In [11]:
Test_db = Test_db.loc[:, ~ Test_db.columns.str.contains('^Unnamed')]

In [12]:
copy_test=copy_test.loc[:, ~ copy_test.columns.str.contains('^Unnamed')]

In [13]:
Train_db = Train_db.loc[:, ~ Train_db.columns.str.contains('^Unnamed')]

In [14]:
Test_db.drop(['String_y'],axis=1,inplace=True)

In [15]:
copy_test.drop(['String_y'],axis=1,inplace=True)

In [16]:
Train_db.drop(['String_y'],axis=1,inplace=True)

In [ ]:
nulls = Train_db.isnull().sum().to_frame()
for index, row in nulls.iterrows():
  print(index, row[0])

Hash 0
String_x 0
Type 0


In [ ]:
#convert into categorical
def category_onehot(Train_db,multicol):
  df_final=Train_db
  i=0
  for j in multicol:
    print(j)
    df1=pd.get_dummies(Train_db[j],drop_first=True)
    Train_db.drop([j],axis=1,inplace=True)
    if i == 0:
        df_final=df1.copy()
    else:
        df_final=pd.concat([df_final,df1],axis=1)
   
    i+=1
    print('Done')
    break
  df_final=pd.concat([final_df,df_final],axis=1)
  
  return df_final

In [ ]:
features=Train_db.select_dtypes(include=['object']).copy()

In [ ]:
Categorical_features= features.columns

In [ ]:
Train_db.head()
Try=Train_db

In [ ]:
X_Train.head()

,Hash,String,Type
0,0581fe38c3900c2a0986adfa3ab2ce68e448a2fd8dd940...,"[!This program cannot be run in DOS mode., Ric...",0
1,0e264c21af77e2e5c779692b27cb3e60280d889619729b...,"[!This program cannot be run in DOS mode., Ric...",0
2,1a6bbd587d62775e6eb70bdbb8da2025684e2d4b470460...,"[!This program cannot be run in DOS mode., Ric...",0
3,60c329aa2b05c0abd71de704b4a068a70e144586246902...,"[!This program cannot be run in DOS mode., Ric...",0
4,9504c3f16641236478091dfec17190e247755eda5a1f57...,"[This program must be run under Win32, .text, ...",0


In [ ]:
len(X_Train['String'][0])

6998

In [ ]:
Train_db=shuffle(Train_db)
Test_db=shuffle(Test_db)

In [ ]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()

# Try['Hash'] = lbl.fit_transform(Try['Hash'].astype(str))
Train_db['Hash'] = lbl.fit_transform(Train_db['Hash'].astype(str))
Test_db['Hash'] = lbl.fit_transform(Test_db['Hash'].astype(str))

In [ ]:
Train_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 2627 to 2808
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Hash      3000 non-null   int64
 1   String_x  3000 non-null   int64
 2   Type      3000 non-null   int64
dtypes: int64(3)
memory usage: 93.8 KB


In [ ]:
# Try['String_x'] = lbl.fit_transform(Try['String_x'].astype(str))
Train_db['String_x'] = lbl.fit_transform(Train_db['String_x'].astype(str))
Test_db['String_x'] = lbl.fit_transform(Test_db['String_x'].astype(str))

In [ ]:
Try.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Hash      2500 non-null   int64
 1   String_x  2500 non-null   int64
 2   Type      2500 non-null   int64
dtypes: int64(3)
memory usage: 78.1 KB


In [ ]:
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder
OneHotEncoder().fit_transform(Train_db)
OneHotEncoder().fit_transform(Test_db)

<2001x3963 sparse matrix of type '<class 'numpy.float64'>'
	with 4002 stored elements in Compressed Sparse Row format>

In [ ]:
Train_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Hash      2500 non-null   object
 1   String_x  2500 non-null   object
 2   Type      2500 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 78.1+ KB


In [ ]:
import xgboost
clsfr=xgboost.XGBClassifier()
X_Train_XGB=Train_db.drop(['Type'],axis=True)
Y_Train_XGB=Train_db['Type']
clsfr.fit(X_Train_XGB,Y_Train_XGB)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
import scipy as sp
!pip install sk-dist
!pip install pyspark
from sklearn.model_selection import train_test_split 
import time
from sklearn import datasets, svm
from skdist.distribute.search import DistGridSearchCV
from pyspark.sql import SparkSession 
# instantiate spark session
spark = (   
    SparkSession    
    .builder    
    .getOrCreate()    
    )
sc = spark.sparkContext 
# th

     |████████████████████████████████| 204.7MB 41kB/s 
     |████████████████████████████████| 204kB 37.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=0d113f18974b9dbf9d92bab2219a2d426a298d1782b71088f62a9ceb21c594ba
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [ ]:
classifier = svm.SVC()
param_grid = {
    "C": [0.01, 0.01, 0.1, 1.0, 10.0, 20.0, 50.0], 
    "gamma": ["scale", "auto", 0.001, 0.01, 0.1], 
    "kernel": ["rbf", "poly", "sigmoid"]
    }
scoring = "f1_weighted"
cv = 10

In [ ]:
# hyperparameter optimization
start = time.time()
model = DistGridSearchCV(    
    classifier, param_grid,     
    sc=sc, cv=cv, scoring=scoring,
    verbose=True    
    )

In [ ]:
X_Train_XGB_for_sk=Train_db['String_x']
X_Train_XGB_for_sk=X_Train_XGB_for_sk.values
Y_Train_XGB=Train_db['Type'].values
df=X_Train_XGB_for_sk


In [ ]:
X_Train_XGB_for_sk.reshape(2500,1)

array([[ 245],
       [ 211],
       [ 472],
       ...,
       [  64],
       [1901],
       [1273]])

In [ ]:
Y_Train_XGB.reshape(1,2500)

array([[0, 0, 0, ..., 0, 0, 0]])

In [ ]:
model.fit(X_Train_XGB_for_sk,Y_Train_XGB)

In [ ]:
import pickle
file_name='Final_modelSTATIC-WITHOUT-STRUCTUREJSON.pkl'
pickle.dump(clsfr,open(file_name,'wb'))

In [ ]:
Test_db.shape

(2001, 2)

In [ ]:
Test_db.head()

,Hash,String_x
0,62,1218
1,152,1188
2,277,1422
3,970,638
4,1488,2149


In [ ]:
y_pred=clsfr.predict(Test_db)

NameError: ignored

In [ ]:
y_pred

NameError: ignored

In [ ]:
predicted_df=pd.DataFrame(y_pred)
datasets=pd.concat([copy_test['Hash'],predicted_df],axis=1)
datasets.columns=['Hash','Type']
datasets.to_csv('final_output-Detect WITHOUT STRUCT STATIC.csv',index=False)

In [ ]:
!ls /content/Static_Analysis_Data/Benign

...


In [ ]:
!cat /content/Static_Analysis_Data/Benign/f9f3db0933269dab9161a296396e199ff0d917c2bcf73d5b9cac841307301909/String.txt


!This program cannot be run in DOS mode.
am F
.text
P`.data
.rdata
`@/4
0@.bss
.idata
.CRT
.tls
5x]N
[^_]
[^_]
UWVS
t$$1
u>;\$
 -> 
[^_]
D$$####
\$)f
UWVS
|$$1
9t$ 
<[^_]
L$$1
UWVS1
t$09
[^_]
UWVS
9\$ t2
D$$9D$,
<[^_]
UWVS
,[^_]
$8^N
$D^N
;_t5
>_t@
UWVS
t$DuH
,[^_]
UWVS
t	9X
D$<9
$ ^N
$h^N
D$(9
[^_]
$h^N
$ ^N
$,^N
$,^N
$t^N
|$D1
$t^N
UWVS
,[^_]
,[^_]
,[^_]
UWVS
\$ u:
l[^_]
;T$D
;T$8
;\$ u
L$89L$0
UWVS
<[^_]
[^_]
UWVS
D$8t;
l$P1
D$89
|$P1
L$89
t$<9
[^_]
[^_]
VUUUVS
D$,1
9T$$
\$(;D$
D$,9D$ 
\[^_]
UWVS
|$01
D$\1
D$,1
l[^_]
D$\1
UWVS
,[^_]
UWVS
[^_]
<[^_]
[^_]
UWVS
[^_]
UWVS
iD$8
|$Ot
\[^_]
UWVS
L[^_]
UWVS
;D$8
9D$<
D$h;D$4
D$Li
l$Tt
9\$,
D$Tv6
D$T9\$,w
D$T+D$P
[^_]
+D$P
D$ 1
|$Dy
Et$$
D$X)
9L$P
9L$T
D$(tF
|$<1
$<!J
|[^_]
D$P<
l$Ht 
UWVS
\[^_]
$d!J
D$<gmon
UWVS
<[^_]
UWVS
r	;A
D$T1
|$Hu#
\$`9X
D$H@
$P^N
;\$8
9D$0
9T$(s
L$09
9D$(
CD$()
D$H@
D$H@t
$X+J
D$T9
|[^_]
|$f)
L$X9L$(
$\^N
UWVS
$D,J
$P^N
$\^N
[^_]
[^_]
$x+J
$z+J
UWVS
,[^_]
UWVS
l$`1
D$,)
tF9(
$T.J
D$P9D$L
|[^_]
D$\f
+:tW
[^_]
-ann
[^

In [ ]:
!cat /content/Static_Analysis_Data/Benign/f9f3db0933269dab9161a296396e199ff0d917c2bcf73d5b9cac841307301909/Structure_Info.txt


----------DOS_HEADER----------

[IMAGE_DOS_HEADER]
0x0        0x0   e_magic:                       0x5A4D    
0x2        0x2   e_cblp:                        0x90      
0x4        0x4   e_cp:                          0x3       
0x6        0x6   e_crlc:                        0x0       
0x8        0x8   e_cparhdr:                     0x4       
0xA        0xA   e_minalloc:                    0x0       
0xC        0xC   e_maxalloc:                    0xFFFF    
0xE        0xE   e_ss:                          0x0       
0x10       0x10  e_sp:                          0xB8      
0x12       0x12  e_csum:                        0x0       
0x14       0x14  e_ip:                          0x0       
0x16       0x16  e_cs:                          0x0       
0x18       0x18  e_lfarlc:                      0x40      
0x1A       0x1A  e_ovno:                        0x0       
0x1C       0x1C  e_res:                         
0x24       0x24  e_oemid:                       0x0       
0x26       0x26

In [ ]:
!ls /content/Static_Analysis_Data/Malware/Backdoor/ec82bdd3797ef08f3b873fbf7a9c3aad06673923dd606322850806be2873b485

String.txt  Structure_Info.txt


In [ ]:
!cat /content/Static_Analysis_Data/Malware/Backdoor/ec82bdd3797ef08f3b873fbf7a9c3aad06673923dd606322850806be2873b485/String.txt

This program must be run under Win32
KERNEL32.DLL
ExitProcess
USER32.DLL
MessageBoxA
kmK9&a
k9L&v|!
H*&E
dev'}
R.0 m)
~4\C
p8DZ
kY9X
w_\P,
CtK1
q|-u
)sS7fZ
iv8d
K%r%
DLd+
b;$[#QA
-3B~
AD[A
pW5N
BPTz
ROS6
7	tC~
av2+
P}z?n3
U/;t
2`3VS
VueUH[,@\
[5Q~
t&3H
bvB#
n(`^$
Op	=
Zwxf
9G^^)V
 [d6
z./"
$l7U
a6C9
[u{\
hqmT
8AL!
suK8S
evBFw
	~LW2
J/Cx#
pK,m!m
h(u2
<m[C
~$Hyv
h#aI
xtW)U5i+
emCy)
s!]M
D/8)
%O%+V
QR+V
rS4G;m&
"^]7i
+,EBt?w
X\*E
TK+)P
}yt9
	)1$
@}2i
ROo'<
0yO#
,S!g
cKt;
d@h'
)`H+
z )R
'wA;ya
7Vx\\(bRK
Z\pV3
3U?]
#8aU
I$dOK
^o!rk3aH
(l-#
uW8l
LF%bZS@
aB5(
}TI?_
'$xB
brZ!
~xdA
{v0~)Mf
eWHslFr
 Y.|
Sj>@*
,HDX
<Nj%6Plt
u,1:f
$nw@
F,YdS
@U%G	I
,$Hs
yR#X1
tm/"
v5%`
e]?Lf_H@
-sF5>
+8v%%
=$jTK
\kc+
"Wl$
O%an8
(Id'eJW
|'^$A
s+WP`I
'\)z
&CGAT0
.iYdr
&</9
H=&0
j;&<
kR,*+
;GNf
mwW9
lG6N
V_qk
7<ix
8&Re
Mui]
4(q1
<pov
acf(g
ojTC
QSQ9`
?dCJ
=(pc
W0cq%J
q!c\
NaJ!4U
}zQ7
YQ-1Z
~+2[
S"} 
Y`v<
}lzA
<oJp%
UYzP
KAFy
m {ae
3P.!3
f~q*
\,onY
-}<y
4#NH`
in L
Nf0G
2bf,4
f;9P
\n{M[l
ng P
MelR%-
>,o2
8%*V
m.P>
rs"D

In [ ]:
!cat /content/Static_Analysis_Data/Malware/Backdoor/ec82bdd3797ef08f3b873fbf7a9c3aad06673923dd606322850806be2873b485/Structure_Info.txt

----------Parsing Warnings----------

Suspicious flags set for section 0. Both IMAGE_SCN_MEM_WRITE and IMAGE_SCN_MEM_EXECUTE are set. This might indicate a packed executable.

Error parsing the import table. AddressOfData overlaps with THUNK_DATA for THUNK at RVA 0x49076

Error parsing the import table. AddressOfData overlaps with THUNK_DATA for THUNK at RVA 0x490a2

----------DOS_HEADER----------

[IMAGE_DOS_HEADER]
0x0        0x0   e_magic:                       0x5A4D    
0x2        0x2   e_cblp:                        0x50      
0x4        0x4   e_cp:                          0x2       
0x6        0x6   e_crlc:                        0x0       
0x8        0x8   e_cparhdr:                     0x4       
0xA        0xA   e_minalloc:                    0xF       
0xC        0xC   e_maxalloc:                    0xFFFF    
0xE        0xE   e_ss:                          0x0       
0x10       0x10  e_sp:                          0xB8      
0x12       0x12  e_csum:                        

In [ ]:
f1_m=open("/content/Static_Analysis_Data/Malware/Backdoor/ec82bdd3797ef08f3b873fbf7a9c3aad06673923dd606322850806be2873b485/String.txt","r")
f2_b=open("/content/Static_Analysis_Data/Benign/f9f3db0933269dab9161a296396e199ff0d917c2bcf73d5b9cac841307301909/String.txt","r")
f3_b=open("/content/Static_Analysis_Data/Benign/f739f93dd1abc55a7d121fefd587d8340c03ba5a38a070c50a4f2649ae67ce23/String.txt","r")
cou=0
for line1 in f2_b:
  for line2 in f3_b:
    if line1==line2:
      cou+=1
print(cou)

0


In [17]:
Test_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Hash      2001 non-null   object
 1   String_x  2001 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [38]:
X_Train.info()
# X_Train=X_Train.sample(frac=1).reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Hash    3000 non-null   object
 1   String  3000 non-null   object
 2   Type    3000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


In [35]:
X2_Train.info()
# X2_Train=X2_Train.sample(frac=1).reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Hash    3000 non-null   object
 1   String  3000 non-null   object
 2   Type    3000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


In [19]:
Test_db.rename(columns = {'String_x':'String'}, inplace = True) 
Test_db.sort_index(inplace=True)
X_Train.sort_index(inplace=True)


In [29]:
B=Train_db['String'].tolist()
T=Test_db['String'].tolist()

In [24]:
Test_db['String'].head()

0    ['Rich', '.text', '`.rdata', '@.data', '.rsrc'...
1    ['Rich', '.text', '`.rdata', '@.data', '.rsrc'...
2    ['.text', 'P`.data', '.rdata', '`@.eh_fram', '...
3    ['McQ<', 'M}Q9', 'M|Q<', ",MQ'", ',qQ<', ',sQ<...
4    ['UPX0', 'UPX1', '.rsrc', '3.08', 'UPX!', 's?p...
Name: String, dtype: object

In [48]:
Benign_match = sum(any(m in L for m in T) for L in B)

In [50]:
Benign_match

3000

In [51]:
Malware_match = sum(any(m in L for m in T) for L in M)

In [52]:
Malware_match

3000

In [1]:
Test_db.to_csv("Save it testing.csv")
Train_db.rename(columns = {'String_x':'String'}, inplace = True) 
Train_db.sort_index(inplace=True)
Train_db.to_csv("save_it_train.csv")

NameError: ignored

In [2]:
import pandas as pd
Train_db=pd.DataFrame()
Test_db=pd.DataFrame()
Test_db=pd.read_csv("Save it testing.csv")
Train_db=pd.read_csv("save_it_train.csv")

In [ ]:
B=Train_db.iloc[:500,:]
B=B['String'].tolist()
T=Test_db['String'].tolist()
import numpy as np
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
df=pd.DataFrame()
df['benignMatchper'] = np.where(fuzz.ratio(T,B) >= 85, "Benign", 'Malware')


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
df

In [ ]:
T.info()
B.info()

In [1]:
df['benignMatchper'] = np.where(fuzz.ratio(T, B) >= 85, "Benign", 'Malware')

NameError: ignored